In [2]:
import requests 
from bs4 import BeautifulSoup as bs 
import pandas as pd
import pprint as pp

# API Authentication 
auth = { 'X-Auth-Token': '732c6cba888e45809a463888b61f0bb3', 'X-Response-Control': 'full'}
matchdays = [17] # Desired Matchweeks
dfs = [] # list of dataframes to be concatenated 

# This is messy. I'll be adding these to csv.
codeMap={'Arsenal FC':'ARS',
'AFC Bournemouth':'BOU',
 'Brighton & Hove Albion':'BHA',
 'Burnley FC':'BUR',
 'Chelsea FC':'CHE',
 'Crystal Palace FC':'CRY',
 'Everton FC':'EVE',
 'Huddersfield Town':'HUD',
 'Leicester City FC':'LEI',
 'Liverpool FC': 'LIV',
 'Manchester City FC':'MCI',
 'Manchester United FC':'MUN',
 'Newcastle United FC':'NEW',
 'Southampton FC':'SOU',
 'Stoke City FC':'STK',
 'Swansea City FC':'SWA',
 'Tottenham Hotspur FC':'TOT',
 'Watford FC': 'WAT',
 'West Ham United FC':'WHU',
 'West Bromwich Albion FC':'WBA'}

# Some League IDs
league_ids =  {445:'premier_league',
                   456:'serie_a',
                   450:'ligue_1',
                   452:'bundesliga',
                   455:'la_liga'}


ids = [445] # Select which ids (leagues) you want those matchdays for
for l_id in ids: # For each desired league
    for i,matchday in enumerate(matchdays): # For each matchday within that league
        url = 'http://api.football-data.org/v1/competitions/{}/leagueTable/?matchday={}'.format(str(l_id),str(matchday)) # Add matchday to URL
        page_source = requests.get(url,headers=auth).json() # Pulling data from API
        # pp.pprint(page_source)

        standings = page_source['standing'] # Data referring to standings
        for team in standings: # For each team within a matchday, within a league
            home = pd.DataFrame(team.pop('home'), index = [0]) # Home Draws/GoalsFor&Against/Wins/Losses
            away = pd.DataFrame(team.pop('away'), index = [0]) # Away ...

            home.columns = ['home'+col.title() for col in home.columns.values] # Change columns to 'homeWins' instead wins.. etc.
            away.columns = ['away'+col.title() for col in away.columns.values] 

            home_and_away = home.merge(away,left_index=True,right_index=True,how='outer') # Merging home and away results to one row
            team.pop('_links') # Removing unecessary data

            # The below dataframe contains all of the data regarding the team, merged with the home/away results
            dfs.append(pd.DataFrame(team, columns=team.keys(),index=[0]).merge( 
                     home_and_away,right_index=True,left_index=True).assign(league=league_ids[l_id],matchday=matchday))
        print('Matchday {} ... Done'.format(i)) # Confirm when complete  
df = pd.concat(dfs) # Vertically concatenate all of the data

df['TEAM CODE'] = df['teamName'].map(codeMap) # Mapping team codes
df.to_csv('test.csv',index=False) 

print(df)



Matchday 0 ... Done
   position                 teamName  \
0         1       Manchester City FC   
0         2     Manchester United FC   
0         3               Chelsea FC   
0         4     Tottenham Hotspur FC   
0         5             Liverpool FC   
0         6               Burnley FC   
0         7               Arsenal FC   
0         8        Leicester City FC   
0         9               Watford FC   
0        10               Everton FC   
0        11           Southampton FC   
0        12        Huddersfield Town   
0        13   Brighton & Hove Albion   
0        14          AFC Bournemouth   
0        15            Stoke City FC   
0        16      Newcastle United FC   
0        17  West Bromwich Albion FC   
0        18        Crystal Palace FC   
0        19       West Ham United FC   
0        20          Swansea City FC   

                                            crestURI  playedGames  points  \
0  https://upload.wikimedia.org/wikipedia/en/e/eb...          